In [1]:
from dotenv import load_dotenv
import os
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_openai import ChatOpenAI

In [2]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY") 
print(api_key[:5])

gsk_4


In [4]:
response_schemas = [
    ResponseSchema(name="destination", description="여행지"),
    ResponseSchema(name="duration", description="여행 기간"),
    ResponseSchema(name="budget", description="여행 예산"),
    ResponseSchema(name="rating", description="여행 만족도 (1-5점)"),
    ResponseSchema(name="activities", description="주요 활동 리스트")
]

parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()

prompt = ChatPromptTemplate.from_template(
    "주어진 여행 후기 또는 계획 텍스트에서 다음 정보를 추출해 JSON으로 반환해줘.\n\n"
    "{format_instructions}\n\n"
    "여행 텍스트:\n{text}"
)

llm = ChatOpenAI(
    api_key=api_key,
    base_url="https://api.groq.com/openai/v1",
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0
)

chain = prompt | llm | parser

travel_review = (
    "지난 주에 부산으로 2박 3일 여행을 다녀왔어요. 총 30만원 정도 썼는데 해운대에서 바다구경하고, "
    "자갈치시장에서 회 먹고, 감천문화마을도 구경했어요. 정말 만족스러운 여행이었습니다. "
    "5점 만점에 4점 정도 줄 수 있을 것 같아요."
)

structured_info = chain.invoke({"text": travel_review, "format_instructions": format_instructions})

print("--- 추출된 여행 정보 ---")
print(f"여행지: {structured_info['destination']}")
print(f"기간: {structured_info['duration']}")
print(f"예산: {structured_info['budget']}")
print(f"추천도: {structured_info['rating']}")
print(f"주요 활동: {''.join(structured_info['activities'])}")

print("\n--- 결과 타입 ---")
print(type(structured_info))

--- 추출된 여행 정보 ---
여행지: 부산
기간: 2박 3일
예산: 30만원
추천도: 4점
주요 활동: 해운대 바다구경, 자갈치시장 회 먹기, 감천문화마을 구경

--- 결과 타입 ---
<class 'dict'>
